In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

from warnings import simplefilter 
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning)

In [2]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/datasets")
#os.chdir("C:/diego_tools/labo3/dataset")
os.chdir("/home/dcastro_contacto/buckets/b1/datasets/")

####################################################

In [3]:
#################################
###### Variables Generales ######
num_lags = 3

In [4]:
arch_sellout_base = "emp3_sellout_base.csv"

In [5]:
df = pd.read_csv(arch_sellout_base)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19639107 entries, 0 to 19639106
Data columns (total 25 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   product_id                 int64  
 1   periodo                    int64  
 2   customer_id                int64  
 3   cero_ventas                int64  
 4   plan_precios_cuidados      float64
 5   cust_request_qty           float64
 6   cust_request_tn            float64
 7   tn                         float64
 8   primer_periodo_prod        int64  
 9   primer_periodo_cliente     int64  
 10  periodo_fecha              object 
 11  mes                        int64  
 12  primer_periodo_fecha_prod  object 
 13  cat1                       object 
 14  cat2                       object 
 15  cat3                       object 
 16  brand                      object 
 17  sku_size                   float64
 18  producto_estrella          float64
 19  temp_media                 float64
 20  

In [7]:
len(df.product_id.unique())

1233

## Nuevos features

In [8]:
# Diferencia entre lo solicitado y lo entregado
df["dif_cust_request_tn"]=df.cust_request_tn-df.tn
df["dif_cust_request_tn_porc"]=np.where(df.cust_request_tn==0,0,100*df.dif_cust_request_tn/df.cust_request_tn)

In [9]:
df[["product_id","customer_id","periodo","tn","cust_request_tn","dif_cust_request_tn","dif_cust_request_tn_porc"]].head(5)

,product_id,customer_id,periodo,tn,cust_request_tn,dif_cust_request_tn,dif_cust_request_tn_porc
0,20001,10001,201701,99.43861,99.43861,0.00000,0.000000
1,20001,10001,201702,198.84365,198.84365,0.00000,0.000000
2,20001,10001,201703,92.46537,92.46537,0.00000,0.000000
3,20001,10001,201704,13.29728,13.29728,0.00000,0.000000
4,20001,10001,201705,101.00563,101.20711,0.20148,0.199077


## Lags (Básicos)

In [10]:
# Lags para product_id-customer_id
for i in range(1, num_lags + 1):
    df[f'tn_lag_{i}'] = df.groupby(['product_id','customer_id'])['tn'].shift(i)

In [11]:
df[["product_id","customer_id","periodo","tn","tn_lag_1","tn_lag_2","tn_lag_3"]].head(50)

,product_id,customer_id,periodo,tn,tn_lag_1,tn_lag_2,tn_lag_3
0,20001,10001,201701,99.43861,NaN,NaN,NaN
1,20001,10001,201702,198.84365,99.43861,NaN,NaN
2,20001,10001,201703,92.46537,198.84365,99.43861,NaN
3,20001,10001,201704,13.29728,92.46537,198.84365,99.43861
4,20001,10001,201705,101.00563,13.29728,92.46537,198.84365
5,20001,10001,201706,128.04792,101.00563,13.29728,92.46537
6,20001,10001,201707,101.20711,128.04792,101.00563,13.29728
7,20001,10001,201708,43.33930,101.20711,128.04792,101.00563
8,20001,10001,201709,289.35024,43.33930,101.20711,128.04792
9,20001,10001,201710,222.11389,289.35024,43.33930,101.20711


## Variable Target

In [12]:
# Variable target
df["tn_mas_2"] = df.groupby(['product_id','customer_id'])['tn'].shift(-2)

In [13]:
df[["product_id","customer_id","periodo","tn","tn_mas_2"]].head(50)

,product_id,customer_id,periodo,tn,tn_mas_2
0,20001,10001,201701,99.43861,92.46537
1,20001,10001,201702,198.84365,13.29728
2,20001,10001,201703,92.46537,101.00563
3,20001,10001,201704,13.29728,128.04792
4,20001,10001,201705,101.00563,101.20711
5,20001,10001,201706,128.04792,43.33930
6,20001,10001,201707,101.20711,289.35024
7,20001,10001,201708,43.33930,222.11389
8,20001,10001,201709,289.35024,111.54944
9,20001,10001,201710,222.11389,131.27150


## Delta Lags (Básicos)

In [14]:
# Delta lags para product_id-customer_id
for i in range(1, num_lags+1):
    df[f'tn_delta_{i}'] = df.groupby(['product_id','customer_id'])['tn'].diff(i)

In [15]:
df[["product_id","customer_id","periodo","tn","tn_lag_1","tn_delta_1","tn_lag_2","tn_delta_2","tn_lag_3","tn_delta_3"]].head(50)

,product_id,customer_id,periodo,tn,tn_lag_1,tn_delta_1,tn_lag_2,tn_delta_2,tn_lag_3,tn_delta_3
0,20001,10001,201701,99.43861,NaN,NaN,NaN,NaN,NaN,NaN
1,20001,10001,201702,198.84365,99.43861,99.40504,NaN,NaN,NaN,NaN
2,20001,10001,201703,92.46537,198.84365,-106.37828,99.43861,-6.97324,NaN,NaN
3,20001,10001,201704,13.29728,92.46537,-79.16809,198.84365,-185.54637,99.43861,-86.14133
4,20001,10001,201705,101.00563,13.29728,87.70835,92.46537,8.54026,198.84365,-97.83802
5,20001,10001,201706,128.04792,101.00563,27.04229,13.29728,114.75064,92.46537,35.58255
6,20001,10001,201707,101.20711,128.04792,-26.84081,101.00563,0.20148,13.29728,87.90983
7,20001,10001,201708,43.33930,101.20711,-57.86781,128.04792,-84.70862,101.00563,-57.66633
8,20001,10001,201709,289.35024,43.33930,246.01094,101.20711,188.14313,128.04792,161.30232
9,20001,10001,201710,222.11389,289.35024,-67.23635,43.33930,178.77459,101.20711,120.90678


## Lags (cat1)

In [16]:
df_cat1 = df.groupby(['cat1','periodo']).agg({'tn':'sum'}).reset_index().sort_values(by=['cat1','periodo'],ascending=True)
df_cat1 = df_cat1.rename(columns={"tn":"cat1_tn"})
df_cat1.head()

,cat1,periodo,cat1_tn
0,FOODS,201701,6499.75930
1,FOODS,201702,7098.70342
2,FOODS,201703,7617.35140
3,FOODS,201704,7285.05226
4,FOODS,201705,7707.92393


In [17]:
# Lags y deltas para cat1
for i in range(1, num_lags + 1):
    df_cat1[f'cat1_tn_lag_{i}'] = df_cat1.groupby('cat1')['cat1_tn'].shift(i)
    df_cat1[f'cat1_tn_delta_{i}'] = df_cat1.groupby('cat1')['cat1_tn'].diff(i)

In [18]:
df_cat1.head()

,cat1,periodo,cat1_tn,cat1_tn_lag_1,cat1_tn_delta_1,cat1_tn_lag_2,cat1_tn_delta_2,cat1_tn_lag_3,cat1_tn_delta_3
0,FOODS,201701,6499.75930,NaN,NaN,NaN,NaN,NaN,NaN
1,FOODS,201702,7098.70342,6499.75930,598.94412,NaN,NaN,NaN,NaN
2,FOODS,201703,7617.35140,7098.70342,518.64798,6499.75930,1117.59210,NaN,NaN
3,FOODS,201704,7285.05226,7617.35140,-332.29914,7098.70342,186.34884,6499.75930,785.29296
4,FOODS,201705,7707.92393,7285.05226,422.87167,7617.35140,90.57253,7098.70342,609.22051


In [19]:
df = df.merge(df_cat1,how="inner",on=["cat1","periodo"])
df.head()

,product_id,periodo,customer_id,cero_ventas,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,primer_periodo_prod,primer_periodo_cliente,...,tn_delta_1,tn_delta_2,tn_delta_3,cat1_tn,cat1_tn_lag_1,cat1_tn_delta_1,cat1_tn_lag_2,cat1_tn_delta_2,cat1_tn_lag_3,cat1_tn_delta_3
0,20001,201701,10001,0,0.0,11.0,99.43861,99.43861,201701,201701,...,NaN,NaN,NaN,20304.28696,NaN,NaN,NaN,NaN,NaN,NaN
1,20001,201701,10002,0,0.0,17.0,38.68301,35.72806,201701,201701,...,NaN,NaN,NaN,20304.28696,NaN,NaN,NaN,NaN,NaN,NaN
2,20001,201701,10003,0,0.0,17.0,143.49426,143.49426,201701,201701,...,NaN,NaN,NaN,20304.28696,NaN,NaN,NaN,NaN,NaN,NaN
3,20001,201701,10004,0,0.0,9.0,184.72927,184.72927,201701,201701,...,NaN,NaN,NaN,20304.28696,NaN,NaN,NaN,NaN,NaN,NaN
4,20001,201701,10005,0,0.0,23.0,19.08407,19.08407,201701,201701,...,NaN,NaN,NaN,20304.28696,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df[["product_id","cat1","periodo","cat1_tn","cat1_tn_lag_1","cat1_tn_delta_1","cat1_tn_lag_2","cat1_tn_delta_2","cat1_tn_lag_3","cat1_tn_delta_3"]].sort_values(by=["product_id","periodo"],ascending=True).head(10)

,product_id,cat1,periodo,cat1_tn,cat1_tn_lag_1,cat1_tn_delta_1,cat1_tn_lag_2,cat1_tn_delta_2,cat1_tn_lag_3,cat1_tn_delta_3
0,20001,HC,201701,20304.28696,NaN,NaN,NaN,NaN,NaN,NaN
1,20001,HC,201701,20304.28696,NaN,NaN,NaN,NaN,NaN,NaN
2,20001,HC,201701,20304.28696,NaN,NaN,NaN,NaN,NaN,NaN
3,20001,HC,201701,20304.28696,NaN,NaN,NaN,NaN,NaN,NaN
4,20001,HC,201701,20304.28696,NaN,NaN,NaN,NaN,NaN,NaN
5,20001,HC,201701,20304.28696,NaN,NaN,NaN,NaN,NaN,NaN
6,20001,HC,201701,20304.28696,NaN,NaN,NaN,NaN,NaN,NaN
7,20001,HC,201701,20304.28696,NaN,NaN,NaN,NaN,NaN,NaN
8,20001,HC,201701,20304.28696,NaN,NaN,NaN,NaN,NaN,NaN
9,20001,HC,201701,20304.28696,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df_cat1_hc = df_cat1[df_cat1.cat1=="HC"]
df_cat1_hc.head()

,cat1,periodo,cat1_tn,cat1_tn_lag_1,cat1_tn_delta_1,cat1_tn_lag_2,cat1_tn_delta_2,cat1_tn_lag_3,cat1_tn_delta_3
36,HC,201701,20304.28696,NaN,NaN,NaN,NaN,NaN,NaN
37,HC,201702,20183.30152,20304.28696,-120.98544,NaN,NaN,NaN,NaN
38,HC,201703,28883.25590,20183.30152,8699.95438,20304.28696,8578.96894,NaN,NaN
39,HC,201704,24453.26459,28883.25590,-4429.99131,20183.30152,4269.96307,20304.28696,4148.97763
40,HC,201705,29339.17333,24453.26459,4885.90874,28883.25590,455.91743,20183.30152,9155.87181


## Lags (cat2)

In [22]:
df_cat2 = df.groupby(['cat2','periodo']).agg({'tn':'sum'}).reset_index().sort_values(by=['cat2','periodo'],ascending=True)
df_cat2 = df_cat2.rename(columns={"tn":"cat2_tn"})
df_cat2.head()

,cat2,periodo,cat2_tn
0,ADEREZOS,201701,4844.34830
1,ADEREZOS,201702,4248.85948
2,ADEREZOS,201703,4543.05532
3,ADEREZOS,201704,4188.45038
4,ADEREZOS,201705,4353.83462


In [23]:
# Lags y deltas para cat2
for i in range(1, num_lags + 1):
    df_cat2[f'cat2_tn_lag_{i}'] = df_cat2.groupby('cat2')['cat2_tn'].shift(i)
    df_cat2[f'cat2_tn_delta_{i}'] = df_cat2.groupby('cat2')['cat2_tn'].diff(i)

In [24]:
df_cat2.head()

,cat2,periodo,cat2_tn,cat2_tn_lag_1,cat2_tn_delta_1,cat2_tn_lag_2,cat2_tn_delta_2,cat2_tn_lag_3,cat2_tn_delta_3
0,ADEREZOS,201701,4844.34830,NaN,NaN,NaN,NaN,NaN,NaN
1,ADEREZOS,201702,4248.85948,4844.34830,-595.48882,NaN,NaN,NaN,NaN
2,ADEREZOS,201703,4543.05532,4248.85948,294.19584,4844.34830,-301.29298,NaN,NaN
3,ADEREZOS,201704,4188.45038,4543.05532,-354.60494,4248.85948,-60.40910,4844.34830,-655.89792
4,ADEREZOS,201705,4353.83462,4188.45038,165.38424,4543.05532,-189.22070,4248.85948,104.97514


In [25]:
df = df.merge(df_cat2,how="inner",on=["cat2","periodo"])
df.head()

,product_id,periodo,customer_id,cero_ventas,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,primer_periodo_prod,primer_periodo_cliente,...,cat1_tn_delta_2,cat1_tn_lag_3,cat1_tn_delta_3,cat2_tn,cat2_tn_lag_1,cat2_tn_delta_1,cat2_tn_lag_2,cat2_tn_delta_2,cat2_tn_lag_3,cat2_tn_delta_3
0,20001,201701,10001,0,0.0,11.0,99.43861,99.43861,201701,201701,...,NaN,NaN,NaN,11153.29607,NaN,NaN,NaN,NaN,NaN,NaN
1,20001,201701,10002,0,0.0,17.0,38.68301,35.72806,201701,201701,...,NaN,NaN,NaN,11153.29607,NaN,NaN,NaN,NaN,NaN,NaN
2,20001,201701,10003,0,0.0,17.0,143.49426,143.49426,201701,201701,...,NaN,NaN,NaN,11153.29607,NaN,NaN,NaN,NaN,NaN,NaN
3,20001,201701,10004,0,0.0,9.0,184.72927,184.72927,201701,201701,...,NaN,NaN,NaN,11153.29607,NaN,NaN,NaN,NaN,NaN,NaN
4,20001,201701,10005,0,0.0,23.0,19.08407,19.08407,201701,201701,...,NaN,NaN,NaN,11153.29607,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df[["product_id","cat2","periodo","cat2_tn","cat2_tn_lag_1","cat2_tn_delta_1","cat2_tn_lag_2","cat2_tn_delta_2","cat2_tn_lag_3","cat2_tn_delta_3"]].sort_values(by=["product_id","periodo"],ascending=True).head(10)

,product_id,cat2,periodo,cat2_tn,cat2_tn_lag_1,cat2_tn_delta_1,cat2_tn_lag_2,cat2_tn_delta_2,cat2_tn_lag_3,cat2_tn_delta_3
0,20001,ROPA LAVADO,201701,11153.29607,NaN,NaN,NaN,NaN,NaN,NaN
1,20001,ROPA LAVADO,201701,11153.29607,NaN,NaN,NaN,NaN,NaN,NaN
2,20001,ROPA LAVADO,201701,11153.29607,NaN,NaN,NaN,NaN,NaN,NaN
3,20001,ROPA LAVADO,201701,11153.29607,NaN,NaN,NaN,NaN,NaN,NaN
4,20001,ROPA LAVADO,201701,11153.29607,NaN,NaN,NaN,NaN,NaN,NaN
5,20001,ROPA LAVADO,201701,11153.29607,NaN,NaN,NaN,NaN,NaN,NaN
6,20001,ROPA LAVADO,201701,11153.29607,NaN,NaN,NaN,NaN,NaN,NaN
7,20001,ROPA LAVADO,201701,11153.29607,NaN,NaN,NaN,NaN,NaN,NaN
8,20001,ROPA LAVADO,201701,11153.29607,NaN,NaN,NaN,NaN,NaN,NaN
9,20001,ROPA LAVADO,201701,11153.29607,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df_cat2_ropa = df_cat2[df_cat2.cat2=="ROPA LAVADO"]
df_cat2_ropa.head()

,cat2,periodo,cat2_tn,cat2_tn_lag_1,cat2_tn_delta_1,cat2_tn_lag_2,cat2_tn_delta_2,cat2_tn_lag_3,cat2_tn_delta_3
360,ROPA LAVADO,201701,11153.29607,NaN,NaN,NaN,NaN,NaN,NaN
361,ROPA LAVADO,201702,10799.07240,11153.29607,-354.22367,NaN,NaN,NaN,NaN
362,ROPA LAVADO,201703,15565.44487,10799.07240,4766.37247,11153.29607,4412.14880,NaN,NaN
363,ROPA LAVADO,201704,12470.81761,15565.44487,-3094.62726,10799.07240,1671.74521,11153.29607,1317.52154
364,ROPA LAVADO,201705,16396.10155,12470.81761,3925.28394,15565.44487,830.65668,10799.07240,5597.02915


## Lags (cat3)

In [28]:
df_cat3 = df.groupby(['cat3','periodo']).agg({'tn':'sum'}).reset_index().sort_values(by=['cat3','periodo'],ascending=True)
df_cat3 = df_cat3.rename(columns={"tn":"cat3_tn"})
df_cat3.head()

,cat3,periodo,cat3_tn
0,ACONDICIONADOR,201701,3550.88234
1,ACONDICIONADOR,201702,3626.58539
2,ACONDICIONADOR,201703,4917.40451
3,ACONDICIONADOR,201704,4793.84217
4,ACONDICIONADOR,201705,5072.35846


In [29]:
# Lags para cat3
for i in range(1, num_lags + 1):
    df_cat3[f'cat3_tn_lag_{i}'] = df_cat3.groupby('cat3')['cat3_tn'].shift(i)
    df_cat3[f'cat3_tn_delta_{i}'] = df_cat3.groupby('cat3')['cat3_tn'].diff(i)

In [30]:
df_cat3.head()

,cat3,periodo,cat3_tn,cat3_tn_lag_1,cat3_tn_delta_1,cat3_tn_lag_2,cat3_tn_delta_2,cat3_tn_lag_3,cat3_tn_delta_3
0,ACONDICIONADOR,201701,3550.88234,NaN,NaN,NaN,NaN,NaN,NaN
1,ACONDICIONADOR,201702,3626.58539,3550.88234,75.70305,NaN,NaN,NaN,NaN
2,ACONDICIONADOR,201703,4917.40451,3626.58539,1290.81912,3550.88234,1366.52217,NaN,NaN
3,ACONDICIONADOR,201704,4793.84217,4917.40451,-123.56234,3626.58539,1167.25678,3550.88234,1242.95983
4,ACONDICIONADOR,201705,5072.35846,4793.84217,278.51629,4917.40451,154.95395,3626.58539,1445.77307


In [31]:
df = df.merge(df_cat3,how="inner",on=["cat3","periodo"])
df.head()

,product_id,periodo,customer_id,cero_ventas,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,primer_periodo_prod,primer_periodo_cliente,...,cat2_tn_delta_2,cat2_tn_lag_3,cat2_tn_delta_3,cat3_tn,cat3_tn_lag_1,cat3_tn_delta_1,cat3_tn_lag_2,cat3_tn_delta_2,cat3_tn_lag_3,cat3_tn_delta_3
0,20001,201701,10001,0,0.0,11.0,99.43861,99.43861,201701,201701,...,NaN,NaN,NaN,3871.38669,NaN,NaN,NaN,NaN,NaN,NaN
1,20001,201701,10002,0,0.0,17.0,38.68301,35.72806,201701,201701,...,NaN,NaN,NaN,3871.38669,NaN,NaN,NaN,NaN,NaN,NaN
2,20001,201701,10003,0,0.0,17.0,143.49426,143.49426,201701,201701,...,NaN,NaN,NaN,3871.38669,NaN,NaN,NaN,NaN,NaN,NaN
3,20001,201701,10004,0,0.0,9.0,184.72927,184.72927,201701,201701,...,NaN,NaN,NaN,3871.38669,NaN,NaN,NaN,NaN,NaN,NaN
4,20001,201701,10005,0,0.0,23.0,19.08407,19.08407,201701,201701,...,NaN,NaN,NaN,3871.38669,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df[["product_id","cat3","periodo","cat3_tn","cat3_tn_lag_1","cat3_tn_delta_1","cat3_tn_lag_2","cat3_tn_delta_2","cat3_tn_lag_3","cat3_tn_delta_3"]].sort_values(by=["product_id","periodo"],ascending=True).head(10)

,product_id,cat3,periodo,cat3_tn,cat3_tn_lag_1,cat3_tn_delta_1,cat3_tn_lag_2,cat3_tn_delta_2,cat3_tn_lag_3,cat3_tn_delta_3
0,20001,Liquido,201701,3871.38669,NaN,NaN,NaN,NaN,NaN,NaN
1,20001,Liquido,201701,3871.38669,NaN,NaN,NaN,NaN,NaN,NaN
2,20001,Liquido,201701,3871.38669,NaN,NaN,NaN,NaN,NaN,NaN
3,20001,Liquido,201701,3871.38669,NaN,NaN,NaN,NaN,NaN,NaN
4,20001,Liquido,201701,3871.38669,NaN,NaN,NaN,NaN,NaN,NaN
5,20001,Liquido,201701,3871.38669,NaN,NaN,NaN,NaN,NaN,NaN
6,20001,Liquido,201701,3871.38669,NaN,NaN,NaN,NaN,NaN,NaN
7,20001,Liquido,201701,3871.38669,NaN,NaN,NaN,NaN,NaN,NaN
8,20001,Liquido,201701,3871.38669,NaN,NaN,NaN,NaN,NaN,NaN
9,20001,Liquido,201701,3871.38669,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df_cat3_liq = df_cat3[df_cat3.cat3=="Liquido"]
df_cat3_liq.head()

,cat3,periodo,cat3_tn,cat3_tn_lag_1,cat3_tn_delta_1,cat3_tn_lag_2,cat3_tn_delta_2,cat3_tn_lag_3,cat3_tn_delta_3
1571,Liquido,201701,3871.38669,NaN,NaN,NaN,NaN,NaN,NaN
1572,Liquido,201702,3908.83266,3871.38669,37.44597,NaN,NaN,NaN,NaN
1573,Liquido,201703,5403.56525,3908.83266,1494.73259,3871.38669,1532.17856,NaN,NaN
1574,Liquido,201704,4242.98976,5403.56525,-1160.57549,3908.83266,334.15710,3871.38669,371.60307
1575,Liquido,201705,5520.31852,4242.98976,1277.32876,5403.56525,116.75327,3908.83266,1611.48586


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19639107 entries, 0 to 19639106
Data columns (total 55 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   product_id                 int64  
 1   periodo                    int64  
 2   customer_id                int64  
 3   cero_ventas                int64  
 4   plan_precios_cuidados      float64
 5   cust_request_qty           float64
 6   cust_request_tn            float64
 7   tn                         float64
 8   primer_periodo_prod        int64  
 9   primer_periodo_cliente     int64  
 10  periodo_fecha              object 
 11  mes                        int64  
 12  primer_periodo_fecha_prod  object 
 13  cat1                       object 
 14  cat2                       object 
 15  cat3                       object 
 16  brand                      object 
 17  sku_size                   float64
 18  producto_estrella          float64
 19  temp_media                 float64
 20  

In [35]:
df.periodo.unique()

array([201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708,
       201709, 201710, 201711, 201712, 201801, 201802, 201803, 201804,
       201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812,
       201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
       201909, 201910, 201911, 201912])

In [36]:
df = df[df.periodo <= 201902]
df = df[df.periodo != 201901]

In [37]:
df.periodo.unique()

array([201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708,
       201709, 201710, 201711, 201712, 201801, 201802, 201803, 201804,
       201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812,
       201902])

In [38]:
df.isna().sum()

product_id                         0
periodo                            0
customer_id                        0
cero_ventas                        0
plan_precios_cuidados              0
cust_request_qty                   0
cust_request_tn                    0
tn                                 0
primer_periodo_prod                0
primer_periodo_cliente             0
periodo_fecha                      0
mes                                0
primer_periodo_fecha_prod          0
cat1                               0
cat2                               0
cat3                               0
brand                              0
sku_size                           0
producto_estrella                  0
temp_media                         0
temp_max_media                     0
temp_min_media                     0
IPC                                0
promedio_mens_dolar_venta          0
catastrofe                         0
dif_cust_request_tn                0
dif_cust_request_tn_porc           0
t

In [39]:
len(df.columns)

55

In [40]:
df.to_csv("emp3_sellout_train.csv", index=False)